## 1. Loading PWS and auxilliary data (primary stations)

### 1.0 Import packages and adjust settings

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import xarray as xr
xr.set_options(display_style='text'); # Show xarray.Dataset representation as text

#import pycomlink as pycml

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 10}) # Set plot font size to 16pt

import numpy as np
import time
from scipy.spatial import cKDTree
from pykrige import OrdinaryKriging as OKpy
import tqdm
import os
import pandas as pd
import matplotlib as mpl

# OPENSENSE Sandbox tools 
# ToDo: Set link from local to github
import PWSpyqcFunctions as pyqc
#import pycomlink as pycml

# Relative path to the data directory
main_dir = r"/home/IWS/seidel/sandbox_abbas/OpenSense/Data"


`PWSpy-qc` specific settings 

In [ ]:
# maximun distance (meters) for which the indicator correlation is calculated
max_distance = 50000

# Precentile threshold for indicator correlation 
prob=0.99

# todo: chekc if and where this is needed
show_plot = True

# minimum number of records (in hours) with values (here 2 months)
min_req_ppt_vals = 2*30*24 

## 1.1 Load the csv data files

In [ ]:
# Automatic weather satation data from KNMI Netherlands
path_primary_network = os.path.join(main_dir, r"data_AWS_stns_data.csv")
path_primary_metadata = os.path.join(main_dir, r"data_AWS_stns_coords.csv")   

# Data from 20 randomly chose radar grid cells as "primary stations 2" for the Amsterdam area
path_primary_network2 = os.path.join(main_dir, r"data_Radar_grid_cell_vals.csv")
path_primary_metadata2 = os.path.join(main_dir, r"data_selected_radar_grid_lonlat.csv") 


# PWS hourly data ()
path_pws_data = os.path.join(main_dir, r"data_AMS_PWS_hourly_shifted.csv")   
path_pws_metadata = os.path.join(main_dir, r"data_AMS_metadata.csv")

In [ ]:
# read primary network 1
print('Reading first primary network data')
# pyqc.read_pcp_csv_file has 'latin-1' encoding in PWSpyqcFunctions.py
in_primary_pcp = pyqc.read_pcp_csv_file(path_to_file=path_primary_network,
                           sep_type=';',
                           index_col=0)

# pyqc uses EPSG:25832, for Netherlands EPSG:28531 is used!
# Reads coordinate file and additionally coverts Lat/Lon to UTM 32
df_prim_coords, prim_coords_xy = pyqc.read_metadata_csv_file(path_primary_metadata,
                                                           sep_type=';', index_col=0,)
df_prim_coords.index = in_primary_pcp.columns
# read primary network 2
print('Reading secondary primary network data')
in_primary_pcp_2 = pyqc.read_pcp_csv_file(path_to_file=path_primary_network2,
                           sep_type=';',
                           index_col=0)

df_prim_coords_2, prim_coords_xy_2 = pyqc.read_metadata_csv_file(
            path_primary_metadata2,
            sep_type=',', index_col=0,)
df_prim_coords_2.index = in_primary_pcp_2.columns

# read pws data
print('Reading PWS data')
df_pws_pcp_hourly = pyqc.read_pcp_csv_file(
        path_to_file=path_pws_data,
                           sep_type=';',
                           index_col=0)
    
df_pws_coords, pws_coords_xy = pyqc.read_metadata_csv_file(
        path_to_file=path_pws_metadata,
                           sep_type=',',
                           index_col=0)
df_pws_coords.index = df_pws_pcp_hourly.columns

### 1.3.1 Location of the stations

In [ ]:
print('Plotting locations of all data')

pyqc.plot_pws_prim_netz_loc(lon_pws=df_pws_coords.lon,
                       lat_pws=df_pws_coords.lat,
                      lon_prim1=df_prim_coords.lon,
                       lat_prim1=df_prim_coords.lat,
                      lon_prim2=df_prim_coords_2.lon,
                       lat_prim2=df_prim_coords_2.lat)

    


In [ ]:
print('Plotting locations of data in and around Amsterdam')

pyqc.plot_pws_prim_netz_loc_AMS(lon_pws=df_pws_coords.lon,
                       lat_pws=df_pws_coords.lat,
                      lon_prim1=df_prim_coords.lon,
                       lat_prim1=df_prim_coords.lat,
                      lon_prim2=df_prim_coords_2.lon,
                       lat_prim2=df_prim_coords_2.lat)

## 2. First PWSpy-QC step: Indicator correlation based filter
The fist QC-filter of PWS-pyqc is based on indicator correlation patterns of the primary network (cf. [Bárdossy et al. 2021](https://doi.org/10.5194/hess-25-583-2021)). First, the data of the primary network are converted to 1 and 0 based on a threshhold, in this case the 99%-percentile. All values above this percentile are 1, all below 0. This indicator correaltion of the primay stations has a spatial structure, i.e. the correlation decreases with increasing distance

In [ ]:
# Plot indicator correlation values different networks
# calculate indicator correlation primary network
print('Calculating indicator correlation')
dist_prim, corr_prim = pyqc.calc_indic_corr_all_stns(
                             coords_stns_xy=prim_coords_xy,
                             pcp_vals=in_primary_pcp.values, prob=0.99)

# calculate indicator correlation second primary network
dist_prim2, corr_prim2 = pyqc.calc_indic_corr_all_stns(
    coords_stns_xy=prim_coords_xy_2,
    pcp_vals=in_primary_pcp_2.values, prob=0.99)

# claculate indicator correlation PWS-PWS
dist_pws, corr_pws = pyqc.calc_indic_corr_all_stns(
                    coords_stns_xy=pws_coords_xy,
                         pcp_vals=df_pws_pcp_hourly.values)

# plot indicator correlations
pyqc.plot_indic_coor(dist_prim=dist_prim,
                corr_prim=corr_prim,
                dist_pws=dist_pws,
               corr_pws=corr_pws)

We can see that there is a large gap up to 20 km in the primary network, hence there's information about the spatial structure missing. This is the reason why the data from the KMNI gauge adjusted radar product are used in addtion

In [ ]:
pyqc.plot_indic_coor(dist_prim=dist_prim,
                corr_prim=corr_prim,
                dist_pws=dist_pws,
               corr_pws=corr_pws,
               dist_prim2=dist_prim2,
               corr_prim2=corr_prim2)

### Applying the Indicator correlation based filter¶

In [ ]:
# apply indicator filter (using primary stations 2)

stn_in = pyqc.indicator_filter(xy_net=pws_coords_xy,
                 prc_net=df_pws_pcp_hourly,
                 xy_dwd=prim_coords_xy_2,
                 prc_dwd=in_primary_pcp_2,
                     prob=0.99, max_distance=max_distance,
                     min_req_ppt_vals=2*24*30, show_plot=True,
                     fn_figure='Indicator Correlation',
                     save_folder=None,
                    tolerance=.99)

# what data type is stn_in
# Was ist tolerance? (1 muss größer als min) Parameter zum aus probieren

In [ ]:
# list the accepted stations
pws_ids_accepted = df_pws_pcp_hourly.columns[np.where(stn_in==True)]
pws_ids_accepted

## 3. Bias Correction

PWS can over- or undererstimate the precepitation compared to a professional refernce station (panel b) in figure below). The reasons herefore are mayfold (c.f. Introduction). The goal is to correct the PWS distribution function usind the distribution function of neighbouring primary network stations. For more details see [Bárdossy et al. (2021)](https://doi.org/10.5194/hess-25-583-2021)




In [ ]:
df_pws_bias_corrected = pyqc.bias_corr_pws_cdf(df_pws_raw=df_pws_pcp_hourly,
                        pws_ids_accepted=pws_ids_accepted,
                        df_pws_coords=df_pws_coords,                            
                        prim_coords_xy_2=prim_coords_xy_2,
                        in_primary_pcp_2=in_primary_pcp_2,
                        df_prim_coords_2=df_prim_coords_2,
                        nstns_correct=len(pws_ids_accepted), #2,
                        plot_figures=False,
                        ppt_min_thr_0_vals=0.1,  # below it all values get p0/2
                        min_qt_to_correct=0.9,)

Umcomment the cell below for saving the output

In [ ]:
#df_pws_bias_corrected.to_csv('PWS_AMS_bias_corr.csv', sep=';')

## 4. Apply On-Event filter 

The goal is to correct the PWS values (false zeros) and (false high values)
[Bárdossy et al. (2021)](https://doi.org/10.5194/hess-25-583-2021)

[El Hachem et at. (2022)](https://doi.org/10.5194/hess-26-6137-2022)

In [ ]:
df_pws_flagged = pyqc.on_event_filter(df_pws=df_pws_bias_corrected, 
                               df_prim=in_primary_pcp_2, 
                               df_pws_coords=df_pws_coords,
                               df_prim_coords=df_prim_coords_2)

Umcomment the cell below for saving the output

In [ ]:
#df_pws_flagged.to_csv('PWS_AMS_bias_corr_event.csv', sep=';')